In [1]:
# Useful Links
# Introduction to Neural Re-Ranking - https://www.youtube.com/watch?v=GSixIsI1eZE

In [2]:
# (base) alepmaros@DESKTOP-BO3QVH2:~/pytorch-playground/datasets/istella-letor/full$ head test.txt -n 1
# 0 qid:000017 1:1 2:4165 3:0 4:9658 5:53 6:343 7:6 8:562 9:0.540929 10:1 11:1 12:1 13:3 14:1 15:0.260698 16:0.086899 17:0.086899 18:0.086899 19:4662 20:4662 21:0 22:1 23:0 24:1 25:1.0 26:0 27:0 28:0 29:0 30:0 31:0 32:0 33:0 34:0 35:0 36:0 37:16 38:0 39:0 40:0 41:0 42:0 43:0 44:0 45:0 46:0 47:0 48:0 49:0 50:0 51:16 52:0 53:0 54:0 55:0 56:0 57:0 58:0 59:0 60:0 61:0 62:0 63:0 64:0 65:16 66:0 67:0 68:0 69:0 70:0 71:0 72:0 73:0 74:0 75:0 76:0 77:0 78:0 79:16 80:0 81:0 82:0 83:0 84:0 85:0 86:0 87:0 88:0 89:0 90:0 91:0 92:0 93:16 94:0 95:0 96:0 97:0 98:0 99:0 100:0 101:0 102:0 103:0 104:0 105:0 106:0 107:16 108:0 109:0 110:0 111:0 112:0 113:0 114:0 115:0 116:0 117:0 118:0 119:0 120:0 121:16 122:0 123:0 124:1.0 125:3.0 126:1.0 127:0.260698 128:0.086899 129:0.086899 130:0.086899 131:4662.0 132:4662.0 133:0.0 134:1.0 135:0.0 136:1.0 137:1.0 138:1 139:3 140:1 141:0.260698 142:0.086899 143:0.086899 144:0.086899 145:4662 146:4662 147:0 148:1 149:0 150:1 151:1.0 152:1 153:3 154:1 155:0.260698 156:0.086899 157:0.086899 158:0.086899 159:4662 160:4662 161:0 162:1 163:0 164:1 165:1.0 166:1 167:3 168:1 169:0.260698 170:0.086899 171:0.086899 172:0.086899 173:4662 174:4662 175:0 176:1 177:0 178:1 179:1.0 180:3.0 181:0.086899 182:3 183:0.086899 184:3 185:3 186:0.086899 187:0.086899 188:2 189:7 190:4 191:1 192:0 193:14 194:4 195:2 196:518674 197:0 198:0 199:0 200:3 201:0 202:39 203:0 204:1 205:0 206:0 207:3 208:0 209:5 210:0 211:150 212:193 213:1 214:0 215:0 216:94 217:1 218:1 219:0 220:2

In [3]:
from IPython.core.display import HTML

display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType
import numpy as np
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

spark = SparkSession.builder.config("spark.driver.memory", "10g").getOrCreate()

24/05/26 19:30:50 WARN Utils: Your hostname, DESKTOP-BO3QVH2 resolves to a loopback address: 127.0.1.1; using 172.20.5.90 instead (on interface eth0)
24/05/26 19:30:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/26 19:30:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
replace_infs_udf = F.udf(lambda x: x if not np.isinf(x) else None, FloatType())

In [6]:
feature_names = []
for ff in ["train", "test"]:
    df = (
        spark.read.option("inferSchema", False)
        .option("delimiter", " ")
        .csv(f"./datasets/istella-letor/full/{ff}.txt")
        .drop("_c222")
    )

    for col in df.columns:
        if col != "_c0":
            df = df.withColumn(col, F.split(col, ":").getItem(1))

        if col not in ["_c0", "_c1"]:
            if ff == "train":
                feature_names.append(col)
            df = df.withColumn(col, replace_infs_udf(F.col(col).cast("float")))

    df = df.fillna(0.0, subset=feature_names)
    df = df.withColumnRenamed("_c0", "label")
    df = df.withColumnRenamed("_c1", "qid")

    df = df.withColumn("label", F.col("label").cast("int"))
    df = df.withColumn("qid", F.col("qid").cast("int"))

    assembler = VectorAssembler(inputCols=feature_names, outputCol="features")
    df = assembler.transform(df)

    if ff == "train":
        scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
        scaler = scaler.fit(df)
        df = scaler.transform(df)
    else:
        df = scaler.transform(df)

    (
        df.select("qid", "label", "scaledFeatures")
        .write.mode("overwrite")
        .partitionBy("qid")
        .parquet(f"./datasets/istella-letor/{ff}_parquet")
    )

24/05/26 19:31:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
# Create Custom Data Loader

In [ ]:
# Pointwise Using RankNet (Binary Cross Entropy)

In [ ]:
# Pairwise Using MarginRankingLoss
# https://pytorch.org/docs/stable/generated/torch.nn.MarginRankingLoss.html#torch.nn.MarginRankingLoss

In [ ]:
# Some ListWise Method